# **NLP2_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Decision Variables:
Let:
x1 = litres of the first component liquid used,
x2 = litres of the second component liquid used,

Objective Function:
Maximize Q = ln(x1*x2) - (x1+2*x2)^(1/3)

Constraints:
1. Ensure the squared difference between the litres of each component and the mean of both is below ten:
   (x1 - (x1 + x2)/2)^2 ≤ 10
   (x2 - (x1 + x2)/2)^2 ≤ 10

2. The total volume of the mixture tank: 
   x1 + x2 ≤ 80

3. Non-negativity constraints:
   x1 ≥ 0
   x2 ≥ 0


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

The objective function for this problem is:

Maximize Q = ln(x1*x2) - (x1+2*x2)^(1/3)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:
1. (x1 - (x1 + x2)/2)^2 ≤ 10
2. (x2 - (x1 + x2)/2)^2 ≤ 10
3. x1 + x2 ≤ 80
4. x1 ≥ 0
5. x2 ≥ 0


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Here's how you can model this problem using Pyomo in Python:

```python
from pyomo.environ import *
import numpy as np

# Create a model
model = ConcreteModel()

# Define variables
model.x1 = Var(bounds=(0,None))
model.x2 = Var(bounds=(0,None))

# Define objective function
def objective(model):
  return np.log(model.x1 * model.x2) - ((model.x1 + 2*model.x2)**(1/3))
model.objective = Objective(rule=objective, sense=maximize)

# Define constraints
def constraint1(model):
  return (model.x1 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint1 = Constraint(rule=constraint1)

def constraint2(model):
  return (model.x2 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint2 = Constraint(rule=constraint2)

def constraint3(model):
  return model.x1 + model.x2 <= 80
model.constraint3 = Constraint(rule=constraint3)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the solution
print("Optimal solution: [x1, x2] = [", model.x1(), ",", model.x2(), "]")
```

Note: Values for x1 and x2 will depend on the solver used and its configurations. In production environment, you want to handle situations where the solver failed, or solution is not feasible. Also, it is crucial to make sure that the values of x1 and x2 are within their prescribed bounds.

Also, numpy is used in the objective, thus the solver should accept non-linear objectives like 'ipopt'.

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.3 MB/s eta 0:00:00


In [14]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [15]:
from pyomo.environ import *
import numpy as np

# Create a model
model = ConcreteModel()

# Define variables
model.x1 = Var(bounds=(0,None))
model.x2 = Var(bounds=(0,None))

# Define objective function
def objective(model):
  return np.log(model.x1 * model.x2) - ((model.x1 + 2*model.x2)**(1/3))
model.objective = Objective(rule=objective, sense=maximize)

# Define constraints
def constraint1(model):
  return (model.x1 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint1 = Constraint(rule=constraint1)

def constraint2(model):
  return (model.x2 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint2 = Constraint(rule=constraint2)

def constraint3(model):
  return model.x1 + model.x2 <= 80
model.constraint3 = Constraint(rule=constraint3)

# Solve the model
solver = SolverFactory('ipopt')
solver.solve(model)

# Print the solution
print("Optimal solution: [x1, x2] = [", model.x1(), ",", model.x2(), "]")

ERROR:pyomo.core:Rule failed when generating expression for Objective objective with index None:
TypeError: loop of ufunc does not support argument 0 of type ProductExpression which has no callable log method
ERROR:pyomo.core:Constructing component 'objective' from data=None failed:
    TypeError: loop of ufunc does not support argument 0 of type ProductExpression which has no callable log method


TypeError: loop of ufunc does not support argument 0 of type ProductExpression which has no callable log method

In [17]:
from pyomo.environ import *
import numpy as np

# Create a model
model = ConcreteModel()

# Define variables
model.x1 = Var(bounds=(0,None))
model.x2 = Var(bounds=(0,None))

# Define objective function
def objective(model):
  return log(model.x1 * model.x2) - ((model.x1 + 2*model.x2)**(1/3))
model.objective = Objective(rule=objective, sense=maximize)

# Define constraints
def constraint1(model):
  return (model.x1 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint1 = Constraint(rule=constraint1)

def constraint2(model):
  return (model.x2 - (model.x1 + model.x2)/2)**2 <= 10
model.constraint2 = Constraint(rule=constraint2)

def constraint3(model):
  return model.x1 + model.x2 <= 80
model.constraint3 = Constraint(rule=constraint3)

# Solve the model
solver = SolverFactory('knitro')
solver.solve(model)

# Print the solution
print("Optimal solution: [x1, x2] = [", model.x1(), ",", model.x2(), "]")

Optimal solution: [x1, x2] = [ 43.16227757303646 , 36.837722362270426 ]


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**